In [1]:
import sqlite3

con = sqlite3.connect('../db/vivino.db')
cursor = con.cursor()

In [92]:
try:
    cursor.execute(
        """
        CREATE VIEW country_priority AS
        SELECT c.name, c.users_count
        FROM countries as c
        ORDER BY c.users_count DESC
        LIMIT 1
    """

    )
    con.commit()
    print("view created")
except Exception as e:
    print("Error:", e)
    con.rollback()  # Rollback changes if an error occurs

view created


In [94]:
cursor.execute(
    """
    SELECT * FROM country_priority
"""
).fetchall()

[('États-Unis', 12273684)]

In [89]:
try:
    cursor.execute(
        """
        CREATE VIEW three_grapes_five_wines AS
        WITH top_grapes AS (
            SELECT g.id AS id, g.name AS grape, g.wines_count AS wines_count
            FROM grapes AS g
            ORDER BY g.wines_count DESC
            LIMIT 3
        ),
        ranked_wines AS (
            SELECT w.name AS wine, w.grape_id, 
                   ROW_NUMBER() OVER(PARTITION BY w.grape_id ORDER BY w.ratings_average DESC) as rn
            FROM wines AS w
        )
        SELECT tg.grape, rw.wine
        FROM top_grapes AS tg
        JOIN ranked_wines AS rw ON tg.id = rw.grape_id
        WHERE rw.rn <= 5
        ORDER BY tg.wines_count DESC, rw.rn
        """
    )
    con.commit()
    print("view created")
except Exception as e:
    print("Error:", e)
    con.rollback()  # Rollback changes if an error occurs

view created


In [90]:
cursor.execute(
    """
    SELECT *
    from three_grapes_five_wines               

"""
).fetchall()

[('Cabernet Sauvignon', 'Cabernet Sauvignon'),
 ('Cabernet Sauvignon', 'IX Estate Red'),
 ('Cabernet Sauvignon', 'Special Selection Cabernet Sauvignon'),
 ('Cabernet Sauvignon', 'Harlan Estate Red'),
 ('Cabernet Sauvignon', 'Book 17 XVII'),
 ('Pinot Noir', 'Cristal Rosé Vinothèque'),
 ('Pinot Noir', 'Batard-Montrachet Grand Cru'),
 ('Pinot Noir', 'Collection'),
 ('Pinot Noir', 'Clos du Mesnil'),
 ('Pinot Noir', 'Montrachet Grand Cru')]

In [88]:
cursor.execute(
    """
    DROP VIEW IF EXISTS three_grapes_five_wines
    """
)
con.commit()